<a href="https://colab.research.google.com/github/valeriarpo/Business-Analytics/blob/main/3_Modelo_Arbol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modelo Arbol Decisión**
0. Se cargan las librerias de trabajo
1.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Se trae el modelo
from sklearn.tree import DecisionTreeClassifier  #prestamo interbibliotecario

#Para llamar los archivos del drive sin tener que subirlos
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


1. Se cargan los datos desde el drive


In [ ]:
nxl = "/content/drive/MyDrive/ANALITICA DE NEGOCIOS/0. SolicitantesCrédito(USD).xlsx"
XDB = pd.read_excel(nxl, sheet_name= 1)
XDB=XDB.dropna()    #Evitar celdas vacias
XDB.head()
#Seleccionamos las variables de interés
XD=XDB[["Edad", "Ingresos", "Egresos", "Monto (EAD)"]]
XD.head()                                         #Se necesita array para que los lea como números
XD=np.array(XD)           #Convierte la tabla en números
XD
yd=XDB[["PreApr"]] ; yd=np.array(yd)       #Variable de Salida - Convierte a la tabla


2. Se procede con la construcción del modelo de Árbol

In [ ]:
mar = DecisionTreeClassifier(criterion="gini", max_depth= 4)
mar.fit(XD,yd)     #Valores de pronóstico sirven para evaluar el modelo
ydp = mar.predict(XD)

#Evaluamos el modelo
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(yd, ydp)
VN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
VP = cm[1][1]
print("VN", VN, "FP", FP, "FN", FN, "VP", VP)

Exactitud = (VP+VN)/(VP+VN+FP+FN)
Exactitud
Sensibilidad = VP/(VP+FN)
Sensibilidad
Especificidad = VN/(VN+FP)
Especificidad

print("Exactitud", Exactitud, "Sensibilidad", Sensibilidad, "Especificidad", Especificidad)


VN 2301 FP 658 FN 644 VP 2239
Exactitud 0.7771311194796303 Sensibilidad 0.7766215747485259 Especificidad 0.7776275768840825


3. Se procede con la visualización del arbol

In [ ]:
from sklearn.tree import export_graphviz      #export graphiz es graficador
from pydotplus import graph_from_dot_data                #Que datos le voy a colocar al arbol
vs= ["Edad", "Ingresos", "Egresos", "Monto (EAD)"]       #Estos nombres aparecen en el arbol

dot_data=export_graphviz(mar, feature_names=vs)         #Asegurar nombres del grafíco
graph=graph_from_dot_data(dot_data)      #Se hace gráfico del Árbol
graph.write_png("arbol.png")            #Se crea la imagén



True

4. Le calculamos los valores de preaprob a los solicitantes nuevos

In [ ]:
XDB2=pd.read_excel(nxl, sheet_name=2)   #Hoja dos es la que no tiene salida
XDB2=XDB2[["Edad", "Ingresos", "Egresos", "Monto (EAD)"]]
XDB2=XDB2.dropna()   #Base de datos solicitantes nuevos

XD2=np.array(XDB2)
ydp2=mar.predict(XD2)
XDB2["PreApr"]=ydp2     #Le pego la columna a la base de datos
XDB2.head()

XDB2.to_excel("NuevosSolicitantes.xlsx")   #Importancia del analísis
len(ydp2)    #2536 solicitantes nuevos
np.sum(ydp2==1)
len(ydp2)-np.sum(ydp2==1)

n=(len(ydp2)-np.sum(ydp2==1))/len(ydp2)
print(n)

**Analísis de Resultados**
En la base de datos se puede observar un total de 5842 solicitantes de credíto dividios en 2959 solicitantes preaprobados (51%) y un total de 2883 solicitantes prenegados (49%). De acuerdo con lo anterior, el modelo de arbol arrojó un indice de exactitud cercano al 77%, así como indices de sensibilidad cercanos al 77% igualmente y que muestran el comportamiento del modelo frente a la clasificación de solicitantes preaprobados. Este mismo porcentaje en promedio se pudo observar frente al indice de especificidad el cual mide el comportamiento del modelo en la clasificación de solicitantes prenegados. Es importante mencionar que un modelo que exhiba valores de exactitud (accuracy) por encima del 75%, es considerado un modelo aceptable para el pronóstico de los datos. Para la base de datos de solicitantes nuevos (2536 solicitantes), se puede observar que el modelo clasificó un total de 1260 solicitantes (49.6 %) en la categoría de preaprobados y un total de 1276 (50.3%)en la categoría de negados. Lo anterior muestra la predominancia del modelo por clasificación de la categoría de prenegados.